In [3]:
import pandas as pd
import sqlite3
import os

# --- 1. SETUP PATHS (The Fix) ---
# "../" means "Go up one folder level"
raw_data_path = '../01_Raw_Data/'
output_folder = '../03_Database' 
db_path = os.path.join(output_folder, 'olist.db')

# Safety Check: Create the folder in the ROOT directory if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"📂 Created missing folder: {output_folder}")

# --- 2. FILE MAPPING ---
files = {
    'olist_orders_dataset.csv': 'orders',
    'olist_order_items_dataset.csv': 'order_items',
    'olist_order_payments_dataset.csv': 'order_payments',
    'olist_order_reviews_dataset.csv': 'order_reviews',
    'olist_products_dataset.csv': 'products',
    'olist_sellers_dataset.csv': 'sellers',
    'olist_customers_dataset.csv': 'customers',
    'olist_geolocation_dataset.csv': 'geolocation'
}

# --- 3. THE ENGINE ---
def ingest_data():
    # Print where we are looking (for debugging)
    print(f"📍 Script running in: {os.getcwd()}")
    print(f"🔎 Looking for data in: {os.path.abspath(raw_data_path)}")
    
    conn = sqlite3.connect(db_path)
    print(f"✅ Connected to database at: {db_path}")

    for csv_file, table_name in files.items():
        file_path = os.path.join(raw_data_path, csv_file)
        
        # Check if CSV actually exists before trying to read
        if not os.path.exists(file_path):
            print(f"⚠️ WARNING: Could not find {csv_file}")
            continue

        try:
            df = pd.read_csv(file_path)
            # Basic Cleaning: Lowercase columns, replace spaces with underscores
            df.columns = [c.lower().replace(' ', '_') for c in df.columns]
            
            # Load to DB
            df.to_sql(table_name, conn, if_exists='replace', index=False)
            print(f"🚀 Loaded {table_name}: {len(df):,} rows")
            
        except Exception as e:
            print(f"❌ Error on {table_name}: {e}")
            
    conn.close()
    print("\n🏁 Ingestion Complete.")

if __name__ == "__main__":
    ingest_data()

📍 Script running in: C:\old_D\Coding\Jupyter_all\Olist_Operations_Project\02_Scripts
🔎 Looking for data in: C:\old_D\Coding\Jupyter_all\Olist_Operations_Project\01_Raw_Data
✅ Connected to database at: ../03_Database\olist.db
🚀 Loaded orders: 99,441 rows
🚀 Loaded order_items: 112,650 rows
🚀 Loaded order_payments: 103,886 rows
🚀 Loaded order_reviews: 99,224 rows
🚀 Loaded products: 32,951 rows
🚀 Loaded sellers: 3,095 rows
🚀 Loaded customers: 99,441 rows
🚀 Loaded geolocation: 1,000,163 rows

🏁 Ingestion Complete.


In [2]:
import pandas as pd
import sqlite3
from scipy import stats
import statsmodels.api as sm

# 1. Connect to the Vault
conn = sqlite3.connect('../03_Database/olist.db')

# 2. Extract the Raw Cohorts (No Averages!)
# We pull two specific categories to fight in the "Arena"
query_cohorts = """
SELECT 
    p.product_category_name,
    JULIANDAY(o.order_delivered_customer_date) - JULIANDAY(o.order_delivered_carrier_date) as transit_days,
    oi.freight_value,
    oi.price
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE 
    o.order_status = 'delivered'
    AND o.order_delivered_customer_date IS NOT NULL
    AND o.order_delivered_carrier_date IS NOT NULL
    -- Filter for our two fighters
    AND p.product_category_name IN ('cama_mesa_banho', 'beleza_saude')
    AND s.seller_state = 'SP' AND c.customer_state = 'SP' -- Control for Geography (SP -> SP only)
"""
# Note: The query above needs the joins for sellers/customers to filter SP->SP. 
# Let's fix the query to include those joins.
query_cohorts_fixed = """
SELECT 
    p.product_category_name,
    JULIANDAY(o.order_delivered_customer_date) - JULIANDAY(o.order_delivered_carrier_date) as transit_days
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
JOIN products p ON oi.product_id = p.product_id
JOIN sellers s ON oi.seller_id = s.seller_id
JOIN customers c ON o.customer_id = c.customer_id
WHERE 
    o.order_status = 'delivered'
    AND o.order_delivered_customer_date IS NOT NULL
    AND p.product_category_name IN ('cama_mesa_banho', 'beleza_saude')
    AND s.seller_state = 'SP' AND c.customer_state = 'SP'
"""

df_cohorts = pd.read_sql_query(query_cohorts_fixed, conn)

# 3. Extract Data for Regression (Delay vs Stars)
query_regression = """
SELECT 
    (JULIANDAY(o.order_delivered_customer_date) - JULIANDAY(o.order_estimated_delivery_date)) as delay_days,
    r.review_score
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
JOIN order_reviews r ON oi.order_id = r.order_id
WHERE 
    o.order_status = 'delivered'
    AND o.order_delivered_customer_date IS NOT NULL
    -- We only care about LATE orders to measure the "Anger Penalty"
    -- Or we can look at all orders to see the curve
"""
df_reg = pd.read_sql_query(query_regression, conn)

print("✅ Data Extracted for Lab Analysis.")
print(f"Cohort Size: {len(df_cohorts)} rows")
print(f"Regression Size: {len(df_reg)} rows")

✅ Data Extracted for Lab Analysis.
Cohort Size: 7670 rows
Regression Size: 110005 rows


In [3]:
# 1. Isolate the two groups
bed_bath = df_cohorts[df_cohorts['product_category_name'] == 'cama_mesa_banho']['transit_days']
beauty = df_cohorts[df_cohorts['product_category_name'] == 'beleza_saude']['transit_days']

# 2. Descriptive Stats (The "Eyeball" Test)
print("--- DESCRIPTIVE STATS (SP -> SP) ---")
print(f"Bed/Bath (n={len(bed_bath)}): Mean={bed_bath.mean():.2f} days, Std={bed_bath.std():.2f}")
print(f"Beauty   (n={len(beauty)}):   Mean={beauty.mean():.2f} days, Std={beauty.std():.2f}")

# 3. The T-Test (The "Gavel")
# equal_var=False because we assume they might have different variances (Welch's t-test)
t_stat, p_val = stats.ttest_ind(bed_bath, beauty, equal_var=False)

print("\n--- INFERENTIAL STATS (T-TEST) ---")
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value:     {p_val:.20f}") # High precision to see if it's 0.000...

if p_val < 0.05:
    print("🚨 RESULT: STATISTICALLY SIGNIFICANT.")
    print("The difference is REAL. Bed/Bath uses a structurally different process.")
else:
    print("RESULT: Not Significant. The difference is just luck/noise.")

--- DESCRIPTIVE STATS (SP -> SP) ---
Bed/Bath (n=4633): Mean=5.97 days, Std=5.16
Beauty   (n=3037):   Mean=4.07 days, Std=4.42

--- INFERENTIAL STATS (T-TEST) ---
T-Statistic: 17.2785
P-Value:     0.00000000000000000000
🚨 RESULT: STATISTICALLY SIGNIFICANT.
The difference is REAL. Bed/Bath uses a structurally different process.


In [6]:
# 1. Clean Data for Regression
# Remove nulls
df_reg_clean = df_reg.dropna()

# 2. Filter for "Late" orders only (Delay > 0) to see the punishment curve
# Or keep all to see the full picture. Let's look at the range -5 to +20 days
df_focus = df_reg_clean[ (df_reg_clean['delay_days'] > -5) & (df_reg_clean['delay_days'] < 30) ]

# 3. Run Simple Linear Regression (Stars ~ Delay)
# Y = Review Score, X = Delay Days
X = sm.add_constant(df_focus['delay_days']) # Adds the intercept
model = sm.OLS(df_focus['review_score'], X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           review_score   R-squared:                       0.285
Model:                            OLS   Adj. R-squared:                  0.285
Method:                 Least Squares   F-statistic:                     7248.
Date:                Thu, 18 Dec 2025   Prob (F-statistic):               0.00
Time:                        17:31:54   Log-Likelihood:                -31752.
No. Observations:               18151   AIC:                         6.351e+04
Df Residuals:                   18149   BIC:                         6.352e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.6500      0.011    340.290      0.0

In [7]:
import pandas as pd
import sqlite3
import numpy as np

# --- 1. CONNECT & EXTRACT ---
# We fetch the raw log needed to reconstruct the timeline
db_path = '../03_Database/olist.db'
conn = sqlite3.connect(db_path)

query = """
SELECT 
    o.order_id,
    o.order_status,
    o.order_purchase_timestamp,
    o.order_approved_at,
    o.order_delivered_carrier_date,
    o.order_delivered_customer_date,
    o.order_estimated_delivery_date,
    c.customer_state
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.order_status = 'delivered'
"""

df = pd.read_sql_query(query, conn)
conn.close()

# --- 2. THE REFINERY (Time Conversions) ---
# Convert strings to datetime objects for math
time_cols = ['order_purchase_timestamp', 'order_approved_at', 
             'order_delivered_carrier_date', 'order_delivered_customer_date', 
             'order_estimated_delivery_date']

for col in time_cols:
    df[col] = pd.to_datetime(df[col])

# --- REQUEST A: "SLOW VS LATE" (Physics vs Psychology) ---
# Actual Duration (Physics)
df['actual_duration'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.total_seconds() / 86400

# Promise Gap (Psychology): Estimate - Actual
# Negative = Late (Delivered AFTER Estimate)
df['promise_gap'] = (df['order_estimated_delivery_date'] - df['order_delivered_customer_date']).dt.total_seconds() / 86400
df['is_late'] = df['promise_gap'] < 0

# --- REQUEST B: THE "STEP FUNCTION" (Bottleneck Detection) ---
# 1. Approval (Purchase -> Approved)
df['step_approval'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 86400
# 2. Handling (Approved -> Carrier)
df['step_handling'] = (df['order_delivered_carrier_date'] - df['order_approved_at']).dt.total_seconds() / 86400
# 3. Transit (Carrier -> Customer)
df['step_transit'] = (df['order_delivered_customer_date'] - df['order_delivered_carrier_date']).dt.total_seconds() / 86400

# --- OUTPUT GENERATION ---

print("\n=== REQUEST A: SLOW VS LATE ===")
late_pct = (df['is_late'].sum() / len(df)) * 100
print(f"Total Late Orders: {late_pct:.1f}%")

print("\n--- Actual Duration Stats (Late vs On-Time) ---")
# We group by 'is_late' to see if late orders are physically slower or just had bad estimates
print(df.groupby('is_late')['actual_duration'].agg(['mean', 'std', 'count']).round(1))


print("\n\n=== REQUEST B: THE STEP FUNCTION (Days) ===")
# We verify where the time is actually spent
steps = ['step_approval', 'step_handling', 'step_transit']
stats_table = df[steps].agg(['mean', 'median', 'std']).T.round(1)
print(stats_table)


print("\n\n=== REQUEST C: THE PARETO CHECK (Top 5 States) ===")
# Where is the pain concentrated?
pareto = df.groupby('customer_state')['step_transit'].agg(['count', 'mean'])
pareto = pareto.sort_values('count', ascending=False).head(5).round(1)
pareto.columns = ['Volume', 'Avg Transit Days']
print(pareto)


=== REQUEST A: SLOW VS LATE ===
Total Late Orders: 8.1%

--- Actual Duration Stats (Late vs On-Time) ---
         mean   std  count
is_late                   
False    10.9   6.2  88644
True     31.5  17.3   7826


=== REQUEST B: THE STEP FUNCTION (Days) ===
               mean  median  std
step_approval   0.4     0.0  0.9
step_handling   2.8     1.8  3.5
step_transit    9.3     7.1  8.8


=== REQUEST C: THE PARETO CHECK (Top 5 States) ===
                Volume  Avg Transit Days
customer_state                          
SP               40493               5.6
RJ               12350              12.0
MG               11354               8.8
RS                5344              12.1
PR                4923               8.8


In [8]:
import pandas as pd
import sqlite3
import numpy as np
from scipy import stats

# --- 1. CONNECT & EXTRACT (The "Wide Table" Join) ---
db_path = '../03_Database/olist.db'
conn = sqlite3.connect(db_path)

# We need a more complex join now to get Product Categories and Seller Locations
query = """
SELECT 
    o.order_id,
    o.order_status,
    o.order_delivered_carrier_date,
    o.order_delivered_customer_date,
    o.order_estimated_delivery_date,
    c.customer_state,
    s.seller_state,
    p.product_category_name
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
-- We assume 1 item per order for the dominant logic (simplification for route analysis)
JOIN order_items oi ON o.order_id = oi.order_id
JOIN sellers s ON oi.seller_id = s.seller_id
JOIN products p ON oi.product_id = p.product_id
WHERE o.order_status = 'delivered'
"""

df = pd.read_sql_query(query, conn)
conn.close()

# --- 2. PREPROCESSING ---
# Convert dates
cols = ['order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
for col in cols:
    df[col] = pd.to_datetime(df[col])

# Calculate Core Metrics
# Transit Time (Carrier -> Customer)
df['transit_days'] = (df['order_delivered_customer_date'] - df['order_delivered_carrier_date']).dt.total_seconds() / 86400

# Late Flag (Actual > Estimate)
df['is_late'] = df['order_delivered_customer_date'] > df['order_estimated_delivery_date']

# Create Route Column
df['route'] = df['seller_state'] + ' -> ' + df['customer_state']

# --- HELPER FUNCTION FOR CONSISTENT REPORTING ---
def get_summary(grouped_df):
    summary = grouped_df.agg({
        'transit_days': ['count', 'mean', 'std'],
        'is_late': 'mean' # Mean of boolean (0/1) gives percentage
    })
    # Flatten columns
    summary.columns = ['Volume', 'Avg Transit', 'Std Dev', 'Late %']
    summary['Late %'] = (summary['Late %'] * 100).round(1)
    summary[['Avg Transit', 'Std Dev']] = summary[['Avg Transit', 'Std Dev']].round(1)
    return summary

# --- REQUEST A: THE ROUTE MATRIX (Geography) ---
print("=== REQUEST A: TOP 10 ROUTES (Geography) ===")
route_stats = get_summary(df.groupby('route'))
top_routes = route_stats.sort_values('Volume', ascending=False).head(10)
print(top_routes)

# T-TEST: SP->SP vs SP->RJ
sp_sp = df[df['route'] == 'SP -> SP']['transit_days'].dropna()
sp_rj = df[df['route'] == 'SP -> RJ']['transit_days'].dropna()

t_stat, p_val = stats.ttest_ind(sp_sp, sp_rj, equal_var=False)
print(f"\n[Statistical Test] SP->SP vs SP->RJ:")
print(f"   Avg Difference: {sp_rj.mean() - sp_sp.mean():.1f} days slower")
print(f"   P-Value: {p_val:.5f} ({'SIGNIFICANT' if p_val < 0.05 else 'NOISE'})")


# --- REQUEST B: THE PHYSICS CHECK (Product) ---
print("\n\n=== REQUEST B: TOP 5 CATEGORIES (Physics) ===")
cat_stats = get_summary(df.groupby('product_category_name'))
top_cats = cat_stats.sort_values('Volume', ascending=False).head(5)
print(top_cats)

# T-TEST: Bed_Bath vs Health_Beauty
bed_bath = df[df['product_category_name'] == 'cama_mesa_banho']['transit_days'].dropna()
health = df[df['product_category_name'] == 'beleza_saude']['transit_days'].dropna()

t_stat, p_val = stats.ttest_ind(bed_bath, health, equal_var=False)
print(f"\n[Statistical Test] Bed_Bath vs Health_Beauty:")
print(f"   Avg Difference: {bed_bath.mean() - health.mean():.1f} days slower")
print(f"   P-Value: {p_val:.5f} ({'SIGNIFICANT' if p_val < 0.05 else 'NOISE'})")


# --- REQUEST C: THE RIO BLACK HOLE (Drill-Down) ---
print("\n\n=== REQUEST C: THE RIO BLACK HOLE (Destination = RJ) ===")
rj_dest = df[df['customer_state'] == 'RJ']
rj_stats = get_summary(rj_dest.groupby('seller_state'))
# Show top 5 origins sending TO Rio
print(rj_stats.sort_values('Volume', ascending=False).head(5))

=== REQUEST A: TOP 10 ROUTES (Geography) ===
          Volume  Avg Transit  Std Dev  Late %
route                                         
SP -> SP   35420          4.7      5.1     6.1
SP -> RJ    9403         12.6     10.7    14.8
SP -> MG    8567          9.0      5.9     6.1
SP -> RS    4133         12.7      9.2     7.2
SP -> PR    3609          9.4      6.2     5.2
PR -> SP    3339          7.6      5.9     3.7
MG -> SP    2900          7.7      5.9     4.1
SP -> SC    2709         12.4      7.7    10.6
SP -> BA    2626         16.3     10.0    14.8
MG -> MG    1669          5.5      5.2     3.1

[Statistical Test] SP->SP vs SP->RJ:
   Avg Difference: 7.9 days slower
   P-Value: 0.00000 (SIGNIFICANT)


=== REQUEST B: TOP 5 CATEGORIES (Physics) ===
                        Volume  Avg Transit  Std Dev  Late %
product_category_name                                       
cama_mesa_banho          10953          9.6      8.2     8.4
beleza_saude              9465          8.9      8.6 

In [9]:
import pandas as pd
import sqlite3
import numpy as np

# --- 1. CONNECT & EXTRACT ---
db_path = '../03_Database/olist.db'
conn = sqlite3.connect(db_path)

# We need Estimated Date, Delivered Date, Freight Value, and Geography
query = """
SELECT 
    o.order_id,
    o.order_estimated_delivery_date,
    o.order_delivered_customer_date,
    o.order_delivered_carrier_date,
    oi.freight_value,
    s.seller_state,
    c.customer_state
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN sellers s ON oi.seller_id = s.seller_id
WHERE o.order_status = 'delivered'
"""

df = pd.read_sql_query(query, conn)
conn.close()

# --- 2. THE REFINERY ---
# Date Conversions
cols = ['order_estimated_delivery_date', 'order_delivered_customer_date', 'order_delivered_carrier_date']
for col in cols:
    df[col] = pd.to_datetime(df[col])

# Feature Engineering
df['route'] = df['seller_state'] + ' -> ' + df['customer_state']

# Metric: The Promise (Estimated Days from Carrier Handoff to Customer)
# Note: Usually Promise is calculated from Purchase, but for Logistics ops we often look at Transit Promise
# Let's stick to the User's "Transit" focus: Delivered - Carrier vs Estimated - Carrier
# actually, "Estimated Delivery" is usually usually from Purchase. Let's assume standard "Purchase to Delivery" for simplicity of "Promise"
# But the prompt asks for "Transit Time" context. Let's use (Estimated - Delivered) as the "Buffer".

# Actual Transit (Carrier -> Customer)
df['actual_transit'] = (df['order_delivered_customer_date'] - df['order_delivered_carrier_date']).dt.total_seconds() / 86400

# The "buffer" or "error" in the promise: (Estimated - Actual)
# Positive = Customer Happy (Arrived before estimate)
# Negative = Customer Angry (Arrived after estimate)
df['promise_gap'] = (df['order_estimated_delivery_date'] - df['order_delivered_customer_date']).dt.total_seconds() / 86400

# The "Promised Duration" (Implied)
# We don't have Purchase date in this specific query, but we have the gap. 
# Let's infer: Promised_Transit = Actual_Transit + Promise_Gap
df['promised_transit'] = df['actual_transit'] + df['promise_gap']
# Note: This is an approximation because 'Estimated' is fixed, 'Actual' varies. 
# Ideally: Promised_Date - Carrier_Date = Promised_Transit. Let's use that.
df['promised_transit_days'] = (df['order_estimated_delivery_date'] - df['order_delivered_carrier_date']).dt.total_seconds() / 86400


# --- REQUEST B: THE PROMISE AUDIT (SP->RJ vs SP->RS) ---
print("=== REQUEST B: THE PROMISE AUDIT ===")
target_routes = ['SP -> RJ', 'SP -> RS']
df_b = df[df['route'].isin(target_routes)].copy()

# We calculate the "Late Rate" for context
df_b['is_late'] = df_b['promise_gap'] < 0

# Aggregation
stats_b = df_b.groupby('route').agg({
    'promised_transit_days': ['mean', 'std'],
    'actual_transit': ['mean', 'std'],
    'promise_gap': ['mean'],
    'is_late': 'mean'
}).round(1)

# Clean up for display
stats_b.columns = ['Avg Promise (Days)', 'Std Promise', 'Avg Actual (Days)', 'Std Actual', 'Avg Safety Buffer', 'Late %']
stats_b['Late %'] = (stats_b['Late %'] * 100).round(1)
print(stats_b)


# --- REQUEST C: THE FREIGHT VALUE CHECK (SP->RJ Only) ---
print("\n\n=== REQUEST C: THE FREIGHT VALUE CHECK (SP -> RJ) ===")
df_rj = df[df['route'] == 'SP -> RJ'].copy()

# Create Quartiles for Freight Value
# q=4 means Quartiles: 0-25% (Low), 25-50%, 50-75%, 75-100% (High)
df_rj['freight_bucket'] = pd.qcut(df_rj['freight_value'], q=4, labels=['1. Low Cost', '2. Med-Low', '3. Med-High', '4. High Cost'])

# Filter for just Low (Bottom 25%) and High (Top 25%) as requested
df_rj_filtered = df_rj[df_rj['freight_bucket'].isin(['1. Low Cost', '4. High Cost'])]

stats_c = df_rj_filtered.groupby('freight_bucket').agg({
    'freight_value': ['min', 'max', 'mean'],
    'actual_transit': ['mean', 'std', 'count']
}).round(1)

stats_c.columns = ['Min Price', 'Max Price', 'Avg Price', 'Avg Transit', 'Std Transit', 'Volume']
print(stats_c)

=== REQUEST B: THE PROMISE AUDIT ===
          Avg Promise (Days)  Std Promise  Avg Actual (Days)  Std Actual  \
route                                                                      
SP -> RJ                23.0          8.0               12.6        10.7   
SP -> RS                26.5          7.0               12.7         9.2   

          Avg Safety Buffer  Late %  
route                                
SP -> RJ               10.4    10.0  
SP -> RS               13.8    10.0  


=== REQUEST C: THE FREIGHT VALUE CHECK (SP -> RJ) ===
                Min Price  Max Price  Avg Price  Avg Transit  Std Transit  \
freight_bucket                                                              
1. Low Cost           0.0       15.2       13.6         11.9         10.1   
2. Med-Low            NaN        NaN        NaN          NaN          NaN   
3. Med-High           NaN        NaN        NaN          NaN          NaN   
4. High Cost         19.8      200.5       33.9         13.3     

C:\Users\User\AppData\Local\Temp\ipykernel_3020\2355539657.py:94: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  stats_c = df_rj_filtered.groupby('freight_bucket').agg({


In [10]:
import pandas as pd
import sqlite3
import numpy as np

# --- 1. CONNECT & EXTRACT ---
db_path = '../03_Database/olist.db'
conn = sqlite3.connect(db_path)

# We extract SP->RJ route data specifically for the simulation
query = """
SELECT 
    o.order_id,
    o.order_estimated_delivery_date,
    o.order_delivered_customer_date,
    o.order_delivered_carrier_date,
    oi.freight_value,
    s.seller_state,
    c.customer_state
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN sellers s ON oi.seller_id = s.seller_id
WHERE 
    o.order_status = 'delivered'
    AND s.seller_state = 'SP' 
    AND c.customer_state = 'RJ'
"""

df = pd.read_sql_query(query, conn)
conn.close()

# --- 2. PREPROCESSING ---
cols = ['order_estimated_delivery_date', 'order_delivered_customer_date', 'order_delivered_carrier_date']
for col in cols:
    df[col] = pd.to_datetime(df[col])

# Current State Metrics
df['actual_transit'] = (df['order_delivered_customer_date'] - df['order_delivered_carrier_date']).dt.total_seconds() / 86400
# Current Late Flag
df['is_late_current'] = df['order_delivered_customer_date'] > df['order_estimated_delivery_date']


# --- REQUEST A: THE BUFFER SIMULATION (+4 Days) ---
print("=== REQUEST A: BUFFER SIMULATION (SP -> RJ) ===")

# Create the Simulation: What if we promised 4 days MORE?
# We add 4 days (4 * 24 * 3600 seconds) to the estimate
df['simulated_estimate_date'] = df['order_estimated_delivery_date'] + pd.Timedelta(days=4)

# Calculate Simulated Late Flag
df['is_late_simulated'] = df['order_delivered_customer_date'] > df['simulated_estimate_date']

# The Comparison Table
sim_results = pd.DataFrame({
    'Metric': ['Late Orders (Count)', 'Late Rate (%)'],
    'Current State (Baseline)': [
        df['is_late_current'].sum(), 
        (df['is_late_current'].mean() * 100).round(1)
    ],
    'Simulated State (+4 Days)': [
        df['is_late_simulated'].sum(), 
        (df['is_late_simulated'].mean() * 100).round(1)
    ]
})

print(sim_results.to_string(index=False))
print(f"\n IMPACT: We saved {df['is_late_current'].sum() - df['is_late_simulated'].sum()} customers from disappointment.")


# --- REQUEST B: THE HONESTY CHECK (High Value) ---
print("\n\n=== REQUEST B: HIGH VALUE BUFFER CHECK ===")

# Filter for High Cost (Top 25% of freight value)
high_cost_thresh = df['freight_value'].quantile(0.75)
df_high = df[df['freight_value'] >= high_cost_thresh].copy()

# Calculate the "Promise" (Estimate - Carrier Date) vs "Actual" (Customer - Carrier Date)
# This tells us how many days we *allocated* for the trip vs how many it *took*
df_high['promised_transit'] = (df_high['order_estimated_delivery_date'] - df_high['order_delivered_carrier_date']).dt.total_seconds() / 86400

# Calculate the Safety Buffer (Promise - Actual)
df_high['safety_buffer'] = df_high['promised_transit'] - df_high['actual_transit']

# Stats
avg_actual = df_high['actual_transit'].mean()
avg_promise = df_high['promised_transit'].mean()
avg_buffer = df_high['safety_buffer'].mean()
std_actual = df_high['actual_transit'].std()

print(f"Segment: High Freight Cost (> ${high_cost_thresh:.2f})")
print(f"Avg Actual Transit:   {avg_actual:.1f} days")
print(f"Avg Promised Transit: {avg_promise:.1f} days")
print(f"--------------------------------")
print(f"Current Safety Buffer: {avg_buffer:.1f} days")
print(f"Transit Variability (Std): {std_actual:.1f} days")

# The Verdict logic
print(f"\n[VERDICT]")
if avg_buffer < std_actual:
    print(f"FAIL. Buffer ({avg_buffer:.1f}) is smaller than Variability ({std_actual:.1f}).")
    print("We are setting ourselves up to fail whenever a minor delay happens.")
else:
    print(f"PASS. Buffer ({avg_buffer:.1f}) covers the Variability ({std_actual:.1f}).")

=== REQUEST A: BUFFER SIMULATION (SP -> RJ) ===
             Metric  Current State (Baseline)  Simulated State (+4 Days)
Late Orders (Count)                    1396.0                     1031.0
      Late Rate (%)                      14.8                       11.0

 IMPACT: We saved 365 customers from disappointment.


=== REQUEST B: HIGH VALUE BUFFER CHECK ===
Segment: High Freight Cost (> $19.75)
Avg Actual Transit:   13.2 days
Avg Promised Transit: 23.6 days
--------------------------------
Current Safety Buffer: 10.3 days
Transit Variability (Std): 11.5 days

[VERDICT]
FAIL. Buffer (10.3) is smaller than Variability (11.5).
We are setting ourselves up to fail whenever a minor delay happens.


In [4]:
import pandas as pd
import sqlite3
import numpy as np
import os

# --- PART 1: THE ARCHITECT (Create the Master File) ---
db_path = '../03_Database/olist.db' # Ensure this matches your folder
csv_path = 'master_logistics_table.csv'

# Check if we need to regenerate the Master File
if not os.path.exists(csv_path):
    print("🏗️ Building the Master Analytical Table (ABT)...")
    conn = sqlite3.connect(db_path)
    
    # "The Great Unification" Query
    query = """
    SELECT 
        o.order_id,
        o.order_status,
        o.order_purchase_timestamp,
        o.order_delivered_carrier_date,
        o.order_delivered_customer_date,
        o.order_estimated_delivery_date,
        c.customer_state,
        s.seller_state,
        p.product_category_name,
        oi.freight_value
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    JOIN sellers s ON oi.seller_id = s.seller_id
    JOIN products p ON oi.product_id = p.product_id
    WHERE o.order_status = 'delivered'
    """
    
    df = pd.read_sql_query(query, conn)
    conn.close()
    
    # Save for future speed (The "Cache")
    df.to_csv(csv_path, index=False)
    print(f"✅ Master Table Saved: {len(df)} rows. (Use this CSV from now on!)")
else:
    print("⚡ Loading Master Table from Cache (CSV)...")
    df = pd.read_csv(csv_path)

# --- PART 2: THE ANALYST (Severity Audit) ---

# Preprocessing
time_cols = ['order_delivered_customer_date', 'order_estimated_delivery_date']
for col in time_cols:
    df[col] = pd.to_datetime(df[col])

# Feature Engineering
df['route'] = df['seller_state'] + ' -> ' + df['customer_state']

# Lateness Logic: (Actual - Estimated)
# Positive = Late, Negative = Early
df['lateness_days'] = (df['order_delivered_customer_date'] - df['order_estimated_delivery_date']).dt.total_seconds() / 86400

# The Severity Buckets
def classify_severity(days):
    if days <= 0:
        return '1. On Time / Early'
    elif days <= 3:
        return '2. Minor Late (1-3 Days)'
    else:
        return '3. Major Late (>3 Days)' # The "Red Zone"

df['lateness_tier'] = df['lateness_days'].apply(classify_severity)

# --- REQUEST A: THE FAILURE SPECTRUM (Top 5 Routes) ---
print("\n=== REQUEST A: FAILURE SPECTRUM (By Severity) ===")
# Filter Top 5 Routes
top_routes = df['route'].value_counts().head(5).index.tolist()
df_top = df[df['route'].isin(top_routes)]

# Create the Matrix
spectrum = pd.crosstab(
    index=df_top['route'],
    columns=df_top['lateness_tier'],
    normalize='index' # Show as %
) * 100

# Formatting
cols_order = ['1. On Time / Early', '2. Minor Late (1-3 Days)', '3. Major Late (>3 Days)']
spectrum = spectrum[cols_order].round(1)
spectrum['Total Volume'] = df_top['route'].value_counts()
print(spectrum.sort_values('Total Volume', ascending=False))


# --- REQUEST B: THE RED ZONE AUTOPSY ---
print("\n\n=== REQUEST B: RED ZONE AUTOPSY (Who caused the catastrophe?) ===")
# Filter ONLY for the Red Zone
df_red = df[df['lateness_tier'] == '3. Major Late (>3 Days)']

# Who is the worst offender? (Seller State)
autopsy = df_red['seller_state'].value_counts().reset_index()
autopsy.columns = ['Seller State', 'Red Zone Volume']
autopsy['% of All Crises'] = (autopsy['Red Zone Volume'] / len(df_red) * 100).round(1)

print(f"Total Major Failures: {len(df_red)}")
print(autopsy.head(5))

# The SP -> RJ Specific Check
sp_rj_red = len(df_red[df_red['route'] == 'SP -> RJ'])
print(f"\n[SP -> RJ] Contribution to the Crisis:")
print(f"Volume: {sp_rj_red} orders")
print(f"Share:  {sp_rj_red / len(df_red) * 100:.1f}% of all Major Delays come from this one route.")

🏗️ Building the Master Analytical Table (ABT)...
✅ Master Table Saved: 110197 rows. (Use this CSV from now on!)

=== REQUEST A: FAILURE SPECTRUM (By Severity) ===
lateness_tier  1. On Time / Early  2. Minor Late (1-3 Days)  \
route                                                         
SP -> SP                     93.9                       2.9   
SP -> RJ                     85.2                       3.2   
SP -> MG                     93.9                       2.4   
SP -> RS                     92.7                       2.0   
SP -> PR                     94.8                       2.1   

lateness_tier  3. Major Late (>3 Days)  Total Volume  
route                                                 
SP -> SP                           3.2         35425  
SP -> RJ                          11.7          9403  
SP -> MG                           3.7          8567  
SP -> RS                           5.2          4134  
SP -> PR                           3.1          3609  


=== REQU

In [5]:
import pandas as pd
import numpy as np
import os

# --- 1. LOAD THE MASTER FILE ---
csv_path = 'master_logistics_table.csv'
if os.path.exists(csv_path):
    print("⚡ Loading Master Table from Cache...")
    df = pd.read_csv(csv_path)
else:
    print("⚠️ CSV Cache missing. Please run the previous 'Great Unification' script first.")
    # (In a real scenario, we'd trigger a rebuild here, but assuming you have it from the last step)

# --- 2. PREPROCESSING & FILTERS ---
# Date Conversion
time_cols = ['order_delivered_customer_date', 'order_estimated_delivery_date']
for col in time_cols:
    df[col] = pd.to_datetime(df[col])

# Logic: Route & Lateness
df['route'] = df['seller_state'] + ' -> ' + df['customer_state']
df['lateness_days'] = (df['order_delivered_customer_date'] - df['order_estimated_delivery_date']).dt.total_seconds() / 86400

# "Red Zone" Definition (> 3 Days Late)
df['is_major_late'] = df['lateness_days'] > 3

# FILTER: Isolate the "Crime Scene" (SP -> RJ)
df_rio = df[df['route'] == 'SP -> RJ'].copy()
print(f"Analyzing {len(df_rio)} orders on the SP -> RJ Route...")


# --- REQUEST A: THE "TAIL OF THE DRAGON" (Severity Check) ---
print("\n=== REQUEST A: HOW BAD IS 'MAJOR LATE'? ===")
# Filter only for the failures
rio_failures = df_rio[df_rio['is_major_late'] == True]

median_delay = rio_failures['lateness_days'].median()
horror_delay = rio_failures['lateness_days'].quantile(0.95)

print(f"Context: {len(rio_failures)} Major Failures detected.")
print(f"Median Delay: {median_delay:.1f} days late")
print(f"95th Percentile (The Horror Story): {horror_delay:.1f} days late")

# Verdict Logic
if median_delay > 5:
    print("VERDICT: CATASTROPHIC. We cannot just 'pad' the estimate. The delay is too long.")
else:
    print("VERDICT: MANAGEABLE. A small buffer increase could fix most of this.")


# --- REQUEST B: THE "INTERSECTION" TEST (Product x Route) ---
print("\n\n=== REQUEST B: PRODUCT x ROUTE INTERACTION ===")
# We want to compare the failure rate of Heavy vs Light items WITHIN Rio

# 1. Identify Top 5 Categories in Rio by Volume
top_cats_rio = df_rio['product_category_name'].value_counts().head(5).index.tolist()
df_rio_top = df_rio[df_rio['product_category_name'].isin(top_cats_rio)]

# 2. Calculate Major Late % per Category
intersection = df_rio_top.groupby('product_category_name').agg({
    'lateness_days': 'count',       # Volume
    'is_major_late': 'mean'         # Failure Rate
}).reset_index()

intersection.columns = ['Category', 'Volume', 'Major Late %']
intersection['Major Late %'] = (intersection['Major Late %'] * 100).round(1)

print(intersection.sort_values('Major Late %', ascending=False))

# --- THE FINAL CHECK (Bed/Bath vs Beauty) ---
# Is there a massive gap?
try:
    bb_fail = intersection[intersection['Category'] == 'cama_mesa_banho']['Major Late %'].values[0]
    hb_fail = intersection[intersection['Category'] == 'beleza_saude']['Major Late %'].values[0]
    
    print(f"\n[The Comparison]")
    print(f"Bed/Bath Failure Rate: {bb_fail}%")
    print(f"Beauty Failure Rate:   {hb_fail}%")
    
    gap = bb_fail - hb_fail
    if gap > 5:
        print(f"CONCLUSION: SURGICAL STRIKE. Bed/Bath is {gap:.1f}% riskier. Embargo it.")
    else:
        print(f"CONCLUSION: BLANKET FIX. The risk is systemic ({gap:.1f}% diff). Fix the whole route.")
except:
    print("Could not compare specific categories (maybe they aren't in Top 5). Check the table above.")

⚡ Loading Master Table from Cache...
Analyzing 9403 orders on the SP -> RJ Route...

=== REQUEST A: HOW BAD IS 'MAJOR LATE'? ===
Context: 1099 Major Failures detected.
Median Delay: 11.7 days late
95th Percentile (The Horror Story): 37.9 days late
VERDICT: CATASTROPHIC. We cannot just 'pad' the estimate. The delay is too long.


=== REQUEST B: PRODUCT x ROUTE INTERACTION ===
                Category  Volume  Major Late %
1          esporte_lazer     577          14.0
0        cama_mesa_banho    1402          13.6
3     relogios_presentes     679          12.2
2       moveis_decoracao     781          10.5
4  utilidades_domesticas     565           6.7
Could not compare specific categories (maybe they aren't in Top 5). Check the table above.


In [6]:
import pandas as pd
import numpy as np
import os
from scipy import stats

# --- 1. LOAD THE EVIDENCE ---
csv_path = 'master_logistics_table.csv'
if os.path.exists(csv_path):
    print("⚡ Loading Master Table from Cache...")
    df = pd.read_csv(csv_path)
else:
    print("⚠️ CSV Cache missing. Please run the 'Great Unification' script first.")
    # Fallback to empty df if missing to prevent crash in explanation
    df = pd.DataFrame()

if not df.empty:
    # --- 2. PREPROCESSING ---
    cols = ['order_delivered_customer_date', 'order_delivered_carrier_date', 'order_estimated_delivery_date']
    for col in cols:
        df[col] = pd.to_datetime(df[col])

    # Feature Engineering
    df['route'] = df['seller_state'] + ' -> ' + df['customer_state']
    df['transit_days'] = (df['order_delivered_customer_date'] - df['order_delivered_carrier_date']).dt.total_seconds() / 86400
    df['lateness_days'] = (df['order_delivered_customer_date'] - df['order_estimated_delivery_date']).dt.total_seconds() / 86400
    df['is_late'] = df['lateness_days'] > 0

    # --- HYPOTHESIS A: THE "RIO CHAOS" THEORY (Variance Test) ---
    print("\n=== HYPOTHESIS A: RIO CHAOS (Variance Test) ===")
    
    # 1. Isolate the populations
    rio_transit = df[df['route'] == 'SP -> RJ']['transit_days'].dropna()
    rs_transit = df[df['route'] == 'SP -> RS']['transit_days'].dropna()

    print(f"Sample Sizes: Rio (n={len(rio_transit)}), RS (n={len(rs_transit)})")
    print(f"Rio Std Dev: {rio_transit.std():.2f} days")
    print(f"RS Std Dev:  {rs_transit.std():.2f} days")

    # 2. Levene's Test (Robust to non-normal distributions)
    # H0: Variances are Equal
    # H1: Variances are Different
    stat, p_val_var = stats.levene(rio_transit, rs_transit)

    print(f"Levene Statistic: {stat:.4f}")
    print(f"P-Value: {p_val_var:.20f}")

    print("\n[VERDICT]")
    if p_val_var < 0.05:
        print("🚨 REJECT NULL. The volatility in Rio is STRUCTURAL.")
        print("Standard Buffers will fail because the variance is statistically distinct.")
    else:
        print("FAIL TO REJECT. The variance is normal. We can fix this with math.")


    # --- HYPOTHESIS B: THE "PREMIUM PARADOX" (Independence Test) ---
    print("\n\n=== HYPOTHESIS B: THE PREMIUM PARADOX (Chi-Square) ===")
    
    # Filter for the Crime Scene (SP -> RJ)
    df_rio = df[df['route'] == 'SP -> RJ'].copy()

    # 1. Create Freight Tiers (High vs Low)
    high_thresh = df_rio['freight_value'].quantile(0.75)
    low_thresh = df_rio['freight_value'].quantile(0.25)
    
    # We compare Top 25% payers vs Bottom 25% payers
    df_rio['pay_tier'] = np.where(df_rio['freight_value'] >= high_thresh, 'High Pay', 
                                  np.where(df_rio['freight_value'] <= low_thresh, 'Low Pay', 'Mid'))
    
    # Filter out the middle to sharpen the contrast
    df_test = df_rio[df_rio['pay_tier'] != 'Mid']

    # 2. Contingency Table
    # Rows: Payment Tier, Cols: Late Status
    contingency = pd.crosstab(df_test['pay_tier'], df_test['is_late'])
    
    print("Contingency Table (Count of Orders):")
    print(contingency)
    
    # Calculate Failure Rates for context
    high_fail = contingency.loc['High Pay', True] / contingency.loc['High Pay'].sum() * 100
    low_fail = contingency.loc['Low Pay', True] / contingency.loc['Low Pay'].sum() * 100
    print(f"\nHigh Pay Late Rate: {high_fail:.1f}%")
    print(f"Low Pay Late Rate:  {low_fail:.1f}%")

    # 3. Chi-Square Test of Independence
    # H0: Late Status is Independent of Payment (Money doesn't help)
    # H1: Late Status depends on Payment (Money buys speed)
    chi2, p_val_chi, dof, expected = stats.chi2_contingency(contingency)

    print(f"\nChi-Square Statistic: {chi2:.4f}")
    print(f"P-Value: {p_val_chi:.4f}")

    print("\n[VERDICT]")
    if p_val_chi > 0.05:
        print("🚨 FAIL TO REJECT NULL. The system is 'Fraudulent'.")
        print("Paying more does NOT statistically improve reliability. The bottleneck ignores price.")
    else:
        print("REJECT NULL. The system works. Paying more reduces lateness.")

else:
    print("Error: Could not load data.")

⚡ Loading Master Table from Cache...

=== HYPOTHESIS A: RIO CHAOS (Variance Test) ===
Sample Sizes: Rio (n=9403), RS (n=4133)
Rio Std Dev: 10.69 days
RS Std Dev:  9.17 days
Levene Statistic: 46.1980
P-Value: 0.00000000001113606700

[VERDICT]
🚨 REJECT NULL. The volatility in Rio is STRUCTURAL.
Standard Buffers will fail because the variance is statistically distinct.


=== HYPOTHESIS B: THE PREMIUM PARADOX (Chi-Square) ===
Contingency Table (Count of Orders):
is_late   False  True 
pay_tier              
High Pay   1975    382
Low Pay    2247    346

High Pay Late Rate: 16.2%
Low Pay Late Rate:  13.3%

Chi-Square Statistic: 7.8436
P-Value: 0.0051

[VERDICT]
REJECT NULL. The system works. Paying more reduces lateness.


In [7]:
import pandas as pd
import sqlite3
import os

# --- 1. CONNECT TO THE WAREHOUSE ---
db_path = '../03_Database/olist.db'

# Robust connection logic
if not os.path.exists(db_path):
    if os.path.exists('olist.db'):
        db_path = 'olist.db'
    else:
        print("⚠️ Database not found. Please ensure the 'Great Unification' step was run.")

conn = sqlite3.connect(db_path)

# --- 2. EXECUTE DELEGATION BRIEF NO. 6 ---
# Objective: Identify Top 500 SKUs for "Project Rio Bypass"
# Logic: Filter for RJ Destination -> Rank by Order Volume

query_vital_500 = """
SELECT 
    p.product_id,
    p.product_category_name,
    COUNT(oi.order_id) AS total_orders_in_rio,
    -- Impact: Revenue at Risk
    ROUND(SUM(oi.price), 2) AS total_revenue_in_rio,
    -- Context: Unit Economics
    ROUND(AVG(oi.price), 2) AS avg_item_price,
    -- Context: Logistics Cost (Current Pain)
    ROUND(AVG(oi.freight_value), 2) AS avg_current_freight_cost
FROM 
    order_items oi
JOIN 
    orders o ON oi.order_id = o.order_id
JOIN 
    products p ON oi.product_id = p.product_id
JOIN 
    customers c ON o.customer_id = c.customer_id
WHERE 
    -- FILTER 1: The Problem Zone (Rio)
    c.customer_state = 'RJ'
    -- FILTER 2: Active Timeline (Post-2017 to capture relevant trends)
    AND o.order_purchase_timestamp >= '2017-01-01'
GROUP BY 
    p.product_id, 
    p.product_category_name
ORDER BY 
    -- PRIORITY: Volume (Stopping the highest number of complaints)
    total_orders_in_rio DESC
LIMIT 500;
"""

print("🚀 Executing 'Project Rio Bypass' Query...")
df_pilot = pd.read_sql_query(query_vital_500, conn)
conn.close()

# --- 3. THE HANDOFF ARTIFACT ---
print(f"✅ Generated Pilot List: {len(df_pilot)} SKUs identified.")
print("\n[Preview: The Top 10 'Must-Stock' Items for Rio]")
# We display the top 10 rows so you can put this screenshot in your slide deck
print(df_pilot.head(10).to_string(index=False))

# Optional: Check the "Bed/Bath" dominance in the pilot list
bb_count = df_pilot[df_pilot['product_category_name'] == 'cama_mesa_banho'].shape[0]
print(f"\n[Strategic Insight]")
print(f"Out of the Top 500 Vital Items, {bb_count} are 'Bed/Bath/Table'.")
print(f"This confirms that the 'Bulky Item' strategy is the correct lever.")

🚀 Executing 'Project Rio Bypass' Query...
✅ Generated Pilot List: 500 SKUs identified.

[Preview: The Top 10 'Must-Stock' Items for Rio]
                      product_id  product_category_name  total_orders_in_rio  total_revenue_in_rio  avg_item_price  avg_current_freight_cost
d1c427060a0f73f6b889a5c7c61f2ac4 informatica_acessorios                  102              14724.53          144.36                     42.51
53b36df67ebb7c41585e8d54d6772e08     relogios_presentes                   75               8617.79          114.90                      6.22
422879e10f46682990de24d770e7f83d     ferramentas_jardim                   72               3993.10           55.46                     16.19
53759a2ecddad2bb87a079a1f1519f73     ferramentas_jardim                   69               3783.60           54.83                     19.35
99a4788cb24856965c36a24e339b6058        cama_mesa_banho                   69               6104.50           88.47                     17.55
368c6c730842d7801